In [ ]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats

import sys; sys.path.insert(1, '/home/alessandro/thesis_material/EURECA_scripts')
import os

import geometry as gm
#import conversion as cnv
from func_statistics import compute_pdf_nbin, slopes_r_p, slopes_r_p_onlysub, slopes_r_p_mix, slope_time, slope_maps
import regression
#from coare_functions import *
import meteo
import lcl
import metpy.calc as mpcalc
from metpy.units import units


import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
#from plotmap import plot_background
from plotdistr import * #fb_distribution_npoint, perc_distribution, scatterplot_fit, hist2d
from pdist_fb_distrib_save_fast import *

import warnings; warnings.filterwarnings("ignore")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
if os.path.exists('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_noWarmPatch_noWest.npy'):
    sea_mask_modified = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_noWarmPatch_noWest.npy')

if os.path.exists('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_new.npy'):
    sea_mask = np.load('/home/alessandro/thesis_material/EURECA_scripts/masks/sea_mask_EU_new.npy')

In [ ]:
# read  data
path = '/saturn/simu_Caribbean/coupled_2km/output/'
files_ls = ['QCLOUD', 'U', 'V', 'W', 'OLR', 'PH', 'PHB', 'QVAPOR', 'PB', 'P', 'T']
files_ls = [name + '.nc' for name in files_ls]
wrf_files = [path+name for name in files_ls]

# open dataset with this line of code e.g. 
# xr.open_dataset(wrf_files[files_ls.index('PB.nc')])

In [ ]:
wrf_hourly = path + 'Caraibes_WRF_3h_surf.nc'
ds_hourly = xr.open_dataset(wrf_hourly)
ds_hourly = ds_hourly.sel(time_counter=slice('2020-02-01T01:30:00.000000000', '2020-02-29T22:30:00.000000000'))

sst_day = ds_hourly['SST'].groupby('time_counter.day').mean()            
sst_day = sst_day.where(sea_mask_modified, drop=False); 
sst_day -= 273.15

lon, lat = sst_day.nav_lon, sst_day.nav_lat

# ta_day = ds_hourly['T2'].groupby('time_counter.day').mean()            
# ta_day = ta_day.where(sea_mask_modified, drop=False); 
# ta_day -= 273.15

#### "buoyancy"
# Dt_day = sst_day - ta_day

q2_day = ds_hourly['Q2'].groupby('time_counter.day').mean()            
q2_day = q2_day.where(sea_mask_modified, drop=False);    # kg/kg



# my_method = False

# if not my_method:
#     # daily variables
#     p0 = 101300.
#     RH_day = q2_day/q_sat(ta_day,p0)
# elif my_method:
#     RH_day = xr.open_dataset('/venus/tesi_ale/data_EU/RH/daily/FEB/RH.nc')
#     RH_day = RH_day['rh2_day'].where(sea_mask_modified, drop=False).groupby('time_counter.day').mean()/100.   


In [ ]:
U, V = xr.open_dataset(wrf_files[files_ls.index('U.nc')]), xr.open_dataset(wrf_files[files_ls.index('V.nc')])
    U = U['U'].sel(time_counter=slice('2020-02-01T01:30:00.000000000', '2020-02-29T22:30:00.000000000'))
    V = V['V'].sel(time_counter=slice('2020-02-01T01:30:00.000000000', '2020-02-29T22:30:00.000000000'))

    U10, V10 = U[:,0,:,:], V[:,0,:,:]    #0.5*(U[:,0,:,:]+U[:,1,:,:]), 0.5*(V[:,0,:,:]+V[:,1,:,:])
    U10, V10 = 0.5*(U10[:,:,0:-1]+U10[:,:,1:]) , 0.5*(V10[:,0:-1,:]+V10[:,1:,:])

    U10, V10 = U10.where(sea_mask_modified, drop=False).values, V10.where(sea_mask_modified, drop=False).values
    ws_10 = (U10**2+V10**2)**0.5


    ## daily mean of wind speed
    ws10_day = np.zeros((29,)+ws_10.shape[1:])
    i = np.arange(29).astype(int)
    for j in i:
        ws10_day[j] = np.nanmean(ws_10[j*8:(j+1)*8], axis=0)

    del ws_10